## Importación de librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow.keras as K
import tensorflow as tf

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import Callback
from keras_preprocessing.image import ImageDataGenerator
from sklearn.metrics import balanced_accuracy_score

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-05-07 21:51:41.940154: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 21:51:41.968586: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 21:51:42.620177: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2024-05-07 21:51:43.452609: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 21:51:43.470138: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 21:51:43.470267: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

A continuación, para una mayor claridad, estableceremos los parámetros que se usarán durante el entrenamiento, como el número de batchs, directorios e hiperparámetros.

In [2]:
# Número de clases. 2 para la binaria.
NUM_CLASSES = 2
TRAIN_DIR = "trainThumbnails/"
# Número de canales de cada imagen
CHANNELS = 3

IMAGE_RESIZE = 256
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'relu'
OBJECTIVE_FUNCTION = K.losses.BinaryCrossentropy()

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 50
EARLY_STOP_PATIENCE = 3

BATCH_SIZE = 32

In [3]:
model = K.Sequential()

inputs = K.Input(shape=(IMAGE_RESIZE, IMAGE_RESIZE, CHANNELS))

baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=inputs)

for layer in baseModel.layers[:170]:
    layer.trainable = False

headModel = baseModel.output
headModel = K.layers.AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = K.layers.Flatten(name="flatten")(headModel)
headModel = K.layers.Dense(256, activation="relu")(headModel)
headModel = K.layers.Dropout(0.5)(headModel)
headModel = K.layers.Dense(1, activation=DENSE_LAYER_ACTIVATION)(headModel)

"""
model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x, (IMAGE_RESIZE, IMAGE_RESIZE))))
model.add(resnet)
model.add(K.layers.GlobalAveragePooling2D())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Flatten())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(2, activation='softmax'))
"""
"""
#Still not talking about our train/test data or any pre-processing.

model = K.Sequential()
"""

model = K.models.Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the training process


2024-05-07 21:51:43.481370: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 21:51:43.481521: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 21:51:43.481652: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
baseModel.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 262, 262,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 128, 128,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 128, 128,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 128, 128,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 130, 130,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 64, 64,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 64, 64,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 64, 64,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 64, 64,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 64, 64,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 64, 64,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 64, 64,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 64, 64,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 64, 64,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 64, 64,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 1,054,720 (4.02 MB)

 Non-trainable params: 22,532,992 (85.96 MB)

### Preparación de los datos
Ahora, una vez definido el modelo y la nueva cabecera, debemos de preparar los datos para su lectura. Para ello, debemos tomar las imágenes en sí y el .csv que contiene los metadatos.

In [5]:
df = pd.read_csv('trainSet.csv')


def create_labels(df, dir):
    # Ruta del directorio que deseas listar
    labels = []

    for nombre_directorio, carpetas, archivos in os.walk(dir):
        for pic in archivos:
            if str(pic)[-4:] == ".bmp":
                el = df.loc[df['image'] == str(pic)[:-4]]
            else:
                el = df.loc[df['image'] == str(pic)]

            if (el['label'].values[0]) in malignant:
                labels.append(1)
            else:
                labels.append(0)

    return labels


In [6]:
#train_labels = create_labels(df, TRAIN_DIR)

In [7]:
malignant = ['squamous_cell_carcinoma', 'melanoma', 'basal_cell_carcinoma', 'melanoma_metastatis',
             'intraepithelial_carcinoma', 'kaposi_sarcoma', 'angiosarcoma']


def append_ext(fn):
    if str(fn)[-4:] != ".png" and str(fn)[-4:] != ".JPG":
        return fn + ".bmp"
    else:
        return fn


# Crear una función para asignar valores
def asignar_etiquetas(valor):
    if valor in malignant:
        return "Malignant"
    else:
        return "Benign"

In [8]:
df['image'] = df['image'].apply(append_ext)
df['bin'] = df['label'].apply(asignar_etiquetas)

traindf = df

In [9]:
display(traindf)

,image,dir,label,dataset,bin
0,ASAN_8196.png,datasets/Asan/dataset/hemangioma/ASAN_8196.png,hemangioma,ASAN,Benign
1,ASAN_10059.png,datasets/Asan/dataset/dermatofibroma/ASAN_1005...,dermatofibroma,ASAN,Benign
2,SEV_248_74.png,datasets/Severance/thumbnails/SEV_248_74.png,pyogenic_granuloma,SEVERANCE,Benign
3,ISIC_0068761.JPG,datasets/ISIC/Melanoma/ISIC_0068761.JPG,melanoma,ISIC,Malignant
4,SEV_062_17.png,datasets/Severance/thumbnails/SEV_062_17.png,squamous_cell_carcinoma,SEVERANCE,Malignant
...,...,...,...,...,...
64876,ISIC_0014898.JPG,datasets/ISIC/Nevus/ISIC_0014898.JPG,nevus,ISIC,Benign
64877,SEV_345_42.png,datasets/Severance/thumbnails/SEV_345_42.png,syringoma,SEVERANCE,Benign
64878,ISIC_0062505.JPG,datasets/ISIC/Melanoma/ISIC_0062505.JPG,melanoma,ISIC,Malignant
64879,SEV_164_8.png,datasets/Severance/thumbnails/SEV_164_8.png,hemangioma,SEVERANCE,Benign


In [10]:
datagen = ImageDataGenerator(rescale=1. / 255., validation_split=0.3)

In [11]:
traindf.to_csv("pruebas.csv")

In [12]:
train_ds = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=TRAIN_DIR,
    x_col="image",
    y_col="bin",
    subset="training",
    batch_size=BATCH_SIZE,
    seed=19,
    shuffle=True,
    class_mode="binary")

valid_ds = datagen.flow_from_dataframe(
    dataframe=traindf,
    directory=TRAIN_DIR,
    x_col="image",
    y_col="bin",
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=19,
    shuffle=True,
    class_mode="binary")

Found 45417 validated image filenames belonging to 2 classes.
Found 19464 validated image filenames belonging to 2 classes.


In [13]:
# Early stopping & checkpointing the best model in ../working dir & restoring that as our model for prediction
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

cb_early_stopper = EarlyStopping(monitor='val_loss', patience=EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath='keras_models/best_rs50.keras', monitor='val_loss', save_best_only=True,
                                  mode='auto')

In [14]:
# Establecemos la función de pérdida y el optimizador a utilizar.
# Cargamos las métricas a utilizar

model.compile(loss=OBJECTIVE_FUNCTION,
              optimizer=K.optimizers.Adam(learning_rate=1e-5),
              metrics=[
                  'recall',
                  'acc',
                  K.metrics.Accuracy
              ]
              )

model.fit(train_ds, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, verbose='auto',
          callbacks=[cb_early_stopper], validation_data=valid_ds,
          shuffle=True)
model.save('keras_models/best_rs50.keras')

Epoch 1/50


/home/cristhian/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715111512.677447   35738 service.cc:145] XLA service 0x7fab18002fd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715111512.677475   35738 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-05-07 21:51:52.787417: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-07 21:51:53.342009: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00

   1/1420 ━━━━━━━━━━━━━━━━━━━━ 4:23:39 11s/step - acc: 0.5312 - accuracy: 0.4062 - loss: 5.7253 - recall: 0.0909

I0000 00:00:1715111517.096651   35738 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1420/1420 ━━━━━━━━━━━━━━━━━━━━ 803s 558ms/step - acc: 0.7554 - accuracy: 0.6728 - loss: 3.2553 - recall: 0.0601 - val_acc: 0.7867 - val_accuracy: 0.7867 - val_loss: 3.4383 - val_recall: 0.0000e+00
Epoch 2/50
1420/1420 ━━━━━━━━━━━━━━━━━━━━ 770s 540ms/step - acc: 0.7805 - accuracy: 0.7480 - loss: 3.3171 - recall: 0.0139 - val_acc: 0.7867 - val_accuracy: 0.7867 - val_loss: 3.4383 - val_recall: 0.0000e+00
Epoch 3/50
1420/1420 ━━━━━━━━━━━━━━━━━━━━ 753s 529ms/step - acc: 0.7523 - accuracy: 0.6607 - loss: 3.2198 - recall: 0.0563 - val_acc: 0.7863 - val_accuracy: 0.7711 - val_loss: 3.3712 - val_recall: 9.6339e-04
Epoch 4/50
1420/1420 ━━━━━━━━━━━━━━━━━━━━ 754s 529ms/step - acc: 0.7458 - accuracy: 0.6230 - loss: 3.0074 - recall: 0.0851 - val_acc: 0.7868 - val_accuracy: 0.7763 - val_loss: 3.2363 - val_recall: 7.2254e-04
Epoch 5/50
1420/1420 ━━━━━━━━━━━━━━━━━━━━ 797s 560ms/step - acc: 0.7250 - accuracy: 0.4908 - loss: 2.4852 - recall: 0.1613 - val_acc: 0.7205 - val_accuracy: 0.0060 - val_loss: 0.6

TypeError: Accuracy.get_config() missing 1 required positional argument: 'self'

In [15]:
train_metrics = model.evaluate('keras_models/best_rs50.keras')


609/609 ━━━━━━━━━━━━━━━━━━━━ 243s 398ms/step - acc: 0.7966 - accuracy: 1.4197e-05 - loss: 0.4662 - recall: 0.1627


In [16]:
model.save('keras_models/best_rs50_norm.keras')  # The file needs to end with the .keras extension


TypeError: Accuracy.get_config() missing 1 required positional argument: 'self'

In [17]:
# Making prediction
y_pred = np.argmax(model.predict(valid_ds), axis=1)

609/609 ━━━━━━━━━━━━━━━━━━━━ 245s 400ms/step


In [ ]:
y_true = []
i = 0
"""for image, label in valid_ds.unbatch():
    y_true.append(int(label[0]))
    i += 1"""
y_true = np.concatenate([y for x, y in valid_ds], axis=0)

print(i)


In [19]:

balanced_acc = balanced_accuracy_score(y_true, y_pred)
print(f"Balanced Accuracy: {balanced_acc:.3f}")

ValueError: Found input variables with inconsistent numbers of samples: [0, 19464]

In [20]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_true, y_pred)
print(conf_matrix)
# Visualizamos la matriz de confusión
plt.figure(figsize=(6, 4))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Matriz de Confusión")
plt.colorbar()
plt.xticks(NUM_CLASSES, NUM_CLASSES, rotation=45)
plt.yticks(NUM_CLASSES, NUM_CLASSES)
plt.xlabel("Etiqueta Predicha")
plt.ylabel("Etiqueta Verdadera")
plt.show()


ValueError: Found input variables with inconsistent numbers of samples: [0, 19464]